This notebook outlines the filtering criteria that was used to focus on the samples of interest

In [1]:
import pandas as pd
import numpy as np
metadata = pd.read_table('../data/oral_metadata.txt', index_col=0)

First remove samples that aren't raw.

In [2]:
idx = list(map(lambda x: 'raw' in x, metadata.index)) 
metadata = metadata.loc[idx]

Now filter out B1/2, D1/2, J8/9, since there were technical issues with those samples

In [3]:
exclusion_set = {'B1', 'B2', 'D1', 'D2', 'J8', 'J9'} 
idx = list(map(lambda x: x not in exclusion_set, metadata['paricipant-timepoint']))
metadata = metadata.loc[idx]

Filter out samples that aren't relevant to the brushing event

In [4]:
metadata = metadata.dropna(subset=['brushing_event'])

Clean up metadata to remove misspellings

In [5]:
metadata = metadata.rename(
    columns={
        'paricipant-timepoint': 'participant-timepoint',                
    }
)

Sort samples by host subject and timepoint

In [6]:
metadata = metadata.sort_values(by=['HostSubject', 'Timepoint#'])

Load biom table and match with the metadata

In [7]:
from biom import load_table
table = load_table('../data/oral_deblurred.biom')
ids_to_keep = lambda val, id_, md: id_ in set(metadata.index)
table.filter(ids_to_keep, axis='sample', inplace=True)

485 x 30 <class 'biom.table.Table'> with 3703 nonzero entries (25% dense)

Filter out OTUs that don't appear in at least 5 samples

In [8]:
filter_fn = lambda val, id_, md: np.sum(val>0) > 5
table.filter(filter_fn, axis='observation', inplace=True)

195 x 30 <class 'biom.table.Table'> with 3177 nonzero entries (54% dense)

Sort the OTU table to make sure that it matches the metadata table

In [9]:
table = table.sort_order(list(metadata.index))

Save both the metadata and the OTU biom table

In [12]:
from biom.util import biom_open
with biom_open('../data/oral_trimmed_deblur.biom', 'w') as f:
    table.to_hdf5(f, 'trimmed')
table.to_dataframe().T.to_csv('../data/oral_trimmed_deblur.txt', sep='\t')
metadata.to_csv('../data/oral_trimmed_metadata.txt', sep='\t')    

In [11]:
table.shape

(195, 30)